In [1]:
from torchvision.datasets import CIFAR10
from ffcv.writer import DatasetWriter
from ffcv.fields import RGBImageField, IntField
from torch.utils.data import Subset
import os


write_path = '/tmp/cifar10_train.beton'


cifar_train = CIFAR10(root='./data', train=True, download=True)


dataset = Subset(cifar_train, range(len(cifar_train)))


writer = DatasetWriter(write_path, {
    'image': RGBImageField(max_resolution=32),
    'label': IntField()
})


writer.from_indexed_dataset(dataset)

/home/amir_ubuntu/.virtualenvs/transformer-model-optimization/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 50000/50000 [00:01<00:00, 34926.00it/s]


In [2]:
from ffcv.loader import Loader, OrderOption
from ffcv.transforms import ToTensor, ToDevice, ToTorchImage, Convert
from ffcv.fields.decoders import IntDecoder, SimpleRGBImageDecoder
import torch
import torchvision.transforms as T

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


beton_path = '/tmp/cifar10_train.beton'


train_loader = Loader(
    beton_path,
    batch_size=128,
    num_workers=4,
    order=OrderOption.RANDOM,
    drop_last=True,
    pipelines={
        'image': [
            SimpleRGBImageDecoder(),
            ToTensor(),
            ToDevice(device),
            ToTorchImage(),
            Convert(torch.float32),
        ],
        'label': [
            IntDecoder(),
            ToTensor(),
            ToDevice(device),
            Convert(torch.long)
        ]
    }
)


In [3]:
!pip install timm

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation 

In [4]:
import timm
import torch.nn as nn


model = timm.create_model('deit_small_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 10)
model = model.to(device)

In [13]:
from ffcv.transforms import RandomResizedCrop, RandomHorizontalFlip
from ffcv.transforms import ToTensor, ToDevice, ToTorchImage, Convert
from ffcv.fields.decoders import IntDecoder, SimpleRGBImageDecoder
from ffcv.loader import Loader, OrderOption
from ffcv.loader import Loader, OrderOption
from ffcv.transforms import ToTensor, ToDevice, ToTorchImage, Cutout
from ffcv.fields.decoders import IntDecoder, RandomResizedCropRGBImageDecoder
from torch import float32


image_pipeline = [
    RandomResizedCropRGBImageDecoder((224, 224)),
    Cutout(16),
    ToTensor(),
    ToTorchImage(),
    Convert(float32),
    ToDevice(device)
]

label_pipeline = [
    IntDecoder(),
    ToTensor(),
    ToDevice(device)
]

train_loader = Loader(
    './data/cifar10_train.beton',
    batch_size=128,
    num_workers=4,
    order=OrderOption.RANDOM,
    drop_last=True,
    pipelines={
        'image': image_pipeline,
        'label': label_pipeline
    }
)


In [15]:
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

optimizer = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.05)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        labels = labels.squeeze(1)  # Убираем размерность 1, превращаем в [128]
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, pred = output.max(1)
        correct += pred.eq(labels).sum().item()
        total += labels.size(0)

    acc = 100. * correct / total
    print(f"Train loss: {total_loss:.3f}, Accuracy: {acc:.2f}%")


Epoch 1: 100%|██████████| 390/390 [1:22:05<00:00, 12.63s/it]


Train loss: 786.403, Accuracy: 24.54%


Epoch 2: 100%|██████████| 390/390 [1:15:29<00:00, 11.61s/it]


Train loss: 664.648, Accuracy: 37.61%


Epoch 3: 100%|██████████| 390/390 [1:18:25<00:00, 12.06s/it]


...

Train loss: 515.303, Accuracy: 52.59%


Epoch 9: 100%|██████████| 390/390 [1:19:19<00:00, 12.20s/it]


Train loss: 509.312, Accuracy: 53.27%


Epoch 10: 100%|██████████| 390/390 [1:16:21<00:00, 11.75s/it]

Train loss: 498.352, Accuracy: 54.11%
